In [1]:
import numpy
import pandas as pd
import sqlite3
import os
from pandas.io import sql
from tables import *
import re
import pysam
import matplotlib
import matplotlib.image as mpimg
import seaborn
import matplotlib.pyplot
%matplotlib inline

In [2]:
def vectorizeSequence(seq):
    # the order of the letters is not arbitrary.
    # Flip the matrix up-down and left-right for reverse compliment
    ltrdict = {'a':[1,0,0,0],'c':[0,1,0,0],'g':[0,0,1,0],'t':[0,0,0,1], 'n':[0,0,0,0]}
    return numpy.array([ltrdict[x] for x in seq])

def Generate_training_and_test_datasets(Gem_events_file_path,ARF_label):
    
    #Make Maize genome
    from Bio import SeqIO
    for record in SeqIO.parse(open('/mnt/Data_DapSeq_Maize/MaizeGenome.fa'),'fasta'):
        if record.id =='1':
            chr1 = record.seq.tostring()
        if record.id =='2':
            chr2 = record.seq.tostring()
        if record.id =='3':
            chr3 = record.seq.tostring()
        if record.id =='4':
            chr4 = record.seq.tostring()
        if record.id =='5':
            chr5 = record.seq.tostring()
        if record.id =='6':
            chr6 = record.seq.tostring()
        if record.id =='7':
            chr7 = record.seq.tostring()
        if record.id =='8':
            chr8 = record.seq.tostring()
        if record.id =='9':
            chr9 = record.seq.tostring()
        if record.id =='10':
            chr10 = record.seq.tostring()

    wholegenome = {'chr1':chr1,'chr2':chr2,'chr3':chr3,'chr4':chr4,'chr5':chr5,'chr6':chr6,'chr7':chr7,'chr8':chr8,'chr9':chr9,'chr10':chr10}
    
    
    rawdata = open(Gem_events_file_path) 
    GEM_events=rawdata.read()
    GEM_events=re.split(',|\t|\n',GEM_events)
    GEM_events=GEM_events[0:(len(GEM_events)-1)] # this is to make sure the reshape step works
    GEM_events= numpy.reshape(GEM_events,(-1,10))
    
    #Build Negative dataset
    import random
    
    Bound_Sequences = []
    for i in range(0,len(GEM_events)):
        Bound_Sequences.append(wholegenome[GEM_events[i][0]][int(GEM_events[i][1]):int(GEM_events[i][2])])
    
    Un_Bound_Sequences = []
    count=0
    while count<len(Bound_Sequences):
        chro = numpy.random.choice(['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10'])
        index = random.randint(1,len(wholegenome[chro]))
        absent=True
        for i in range(len(GEM_events)):
            if chro == GEM_events[i][0]:
                if index>int(GEM_events[i][1]) and index<int(GEM_events[i][2]):
                    absent = False
        if absent:
            if wholegenome[chro][index:(index+201)].upper().count('R') == 0 and wholegenome[chro][index:(index+201)].upper().count('W') == 0 and wholegenome[chro][index:(index+201)].upper().count('M') == 0 and wholegenome[chro][index:(index+201)].upper().count('S') == 0 and wholegenome[chro][index:(index+201)].upper().count('K') == 0 and wholegenome[chro][index:(index+201)].upper().count('Y') == 0and wholegenome[chro][index:(index+201)].upper().count('B') == 0 and wholegenome[chro][index:(index+201)].upper().count('D') == 0and wholegenome[chro][index:(index+201)].upper().count('H') == 0 and wholegenome[chro][index:(index+201)].upper().count('V') == 0 and wholegenome[chro][index:(index+201)].upper().count('Z') == 0 and wholegenome[chro][index:(index+201)].upper().count('N') == 0 :
                Un_Bound_Sequences.append(wholegenome[chro][index:(index+201)])
                count=count+1
    response = [0]*(len(Un_Bound_Sequences))
    temp3 = numpy.array(Un_Bound_Sequences)
    temp2 = numpy.array(response)
    neg = pd.DataFrame({'sequence':temp3,'response':temp2})
    
    #Build Positive dataset labeled with signal value
    Bound_Sequences = []
    Responses=[]
    for i in range(0,len(GEM_events)):
        Bound_Sequences.append(wholegenome[GEM_events[i][0]][int(GEM_events[i][1]):int(GEM_events[i][2])])
        Responses.append(float(GEM_events[i][6]))
    
    d = {'sequence' : pd.Series(Bound_Sequences, index=range(len(Bound_Sequences))),
        'response' : pd.Series(Responses, index=range(len(Bound_Sequences)))}
    pos = pd.DataFrame(d)
    
    #Put positive and negative datasets together
    
    LearningData = neg.append(pos)
    LearningData = LearningData.reindex()
    
    #one hot encode sequence data
    counter2=0
    LearningData_seq_OneHotEncoded =numpy.empty([len(LearningData),201,4])

    for counter1 in LearningData['sequence']:
        LearningData_seq_OneHotEncoded[counter2]=vectorizeSequence(counter1.lower())
        counter2=counter2+1
    
    #Create training and test datasets
    from sklearn.cross_validation import train_test_split
    sequence_train, sequence_test, response_train, response_test = train_test_split(LearningData_seq_OneHotEncoded, LearningData['response'], test_size=0.2, random_state=42)
    
    #Saving datasets
    
    numpy.save('/mnt/Data_DapSeq_Maize/'+ARF_label+'_seq_train.npy',sequence_train)
    numpy.save('/mnt/Data_DapSeq_Maize/'+ARF_label+'_res_train.npy',response_train)
    numpy.save('/mnt/Data_DapSeq_Maize/'+ARF_label+'_seq_test.npy',sequence_test)
    numpy.save('/mnt/Data_DapSeq_Maize/'+ARF_label+'_res_test.npy',response_test)
    
def Generate_training_and_test_datasets_no_negative(Gem_events_file_path,ARF_label):
    
    #Make Maize genome
    from Bio import SeqIO
    for record in SeqIO.parse(open('/mnt/Data_DapSeq_Maize/MaizeGenome.fa'),'fasta'):
        if record.id =='1':
            chr1 = record.seq.tostring()
        if record.id =='2':
            chr2 = record.seq.tostring()
        if record.id =='3':
            chr3 = record.seq.tostring()
        if record.id =='4':
            chr4 = record.seq.tostring()
        if record.id =='5':
            chr5 = record.seq.tostring()
        if record.id =='6':
            chr6 = record.seq.tostring()
        if record.id =='7':
            chr7 = record.seq.tostring()
        if record.id =='8':
            chr8 = record.seq.tostring()
        if record.id =='9':
            chr9 = record.seq.tostring()
        if record.id =='10':
            chr10 = record.seq.tostring()

    wholegenome = {'chr1':chr1,'chr2':chr2,'chr3':chr3,'chr4':chr4,'chr5':chr5,'chr6':chr6,'chr7':chr7,'chr8':chr8,'chr9':chr9,'chr10':chr10}
    
    
    rawdata = open(Gem_events_file_path) 
    GEM_events=rawdata.read()
    GEM_events=re.split(',|\t|\n',GEM_events)
    GEM_events=GEM_events[0:(len(GEM_events)-1)] # this is to make sure the reshape step works
    GEM_events= numpy.reshape(GEM_events,(-1,10))
    
    #Build Positive dataset labeled with signal value
    Bound_Sequences = []
    Responses=[]
    for i in range(0,len(GEM_events)):
        Bound_Sequences.append(wholegenome[GEM_events[i][0]][int(GEM_events[i][1]):int(GEM_events[i][2])])
        Responses.append(float(GEM_events[i][6]))
    
    d = {'sequence' : pd.Series(Bound_Sequences, index=range(len(Bound_Sequences))),
        'response' : pd.Series(Responses, index=range(len(Bound_Sequences)))}
    pos = pd.DataFrame(d)
        
    LearningData = pos
    
    #one hot encode sequence data
    counter2=0
    LearningData_seq_OneHotEncoded =numpy.empty([len(LearningData),201,4])

    for counter1 in LearningData['sequence']:
        LearningData_seq_OneHotEncoded[counter2]=vectorizeSequence(counter1.lower())
        counter2=counter2+1
    
    #Create training and test datasets
    from sklearn.cross_validation import train_test_split
    sequence_train, sequence_test, response_train, response_test = train_test_split(LearningData_seq_OneHotEncoded, LearningData['response'], test_size=0.2, random_state=42)
    
    #Saving datasets
    
    numpy.save('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_seq_train.npy',sequence_train)
    numpy.save('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_res_train.npy',response_train)
    numpy.save('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_seq_test.npy',sequence_test)
    numpy.save('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_res_test.npy',response_test)   
    

In [3]:
def Train_and_save_DanQ_model(ARF_label,number_backpropagation_cycles):
    
    #Loading the data
    sequence_train=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'_seq_train.npy')
    response_train=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'_res_train.npy')
    sequence_test=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'_seq_test.npy')
    response_test=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'_res_test.npy')
    
    #Setting up the model
    import keras
    import numpy as np
    from keras import backend
    backend._BACKEND="theano"
    
    #DanQ model

    from keras.preprocessing import sequence
    from keras.optimizers import RMSprop
    from keras.models import Sequential
    from keras.layers.core import Dense
    from keras.layers.core import Merge
    from keras.layers.core import Dropout
    from keras.layers.core import Activation
    from keras.layers.core import Flatten
    from keras.layers.convolutional import Convolution1D, MaxPooling1D
    from keras.regularizers import l2, activity_l1
    from keras.constraints import maxnorm
    from keras.layers.recurrent import LSTM, GRU
    from keras.callbacks import ModelCheckpoint, EarlyStopping
    from keras.layers import Bidirectional

    model = Sequential()
    model.add(Convolution1D(nb_filter=20,filter_length=26,input_dim=4,input_length=201,border_mode="valid")) 
    model.add(Activation('relu'))


    model.add(MaxPooling1D(pool_length=6, stride=6))

    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(5))) 
    model.add(Dropout(0.5))


    model.add(Dense(10))
    model.add(Activation('relu'))

    model.add(Dense(1))

    #compile the model
    model.compile(loss='mean_squared_error', optimizer='rmsprop')
    
    model.fit(sequence_train, response_train, validation_split=0.2,batch_size=100, nb_epoch=number_backpropagation_cycles, verbose=1)
    
    #evaulting correlation between model and test data
    import scipy

    correlation = scipy.stats.pearsonr(response_test,model.predict(sequence_test).flatten())
    correlation_2 = (correlation[0]**2)*100

    print('Percent of variability explained by model: '+str(correlation_2))
    
    # saving the model
    model.save('/mnt/Data_DapSeq_Maize/TrainedModel_DanQ_' +ARF_label+'.h5')

In [4]:
def Train_and_save_DanQ_model_no_negative(ARF_label,number_backpropagation_cycles,train_size):
    
    #Loading the data
    sequence_train=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_seq_train.npy')
    response_train=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_res_train.npy')
    sequence_train=sequence_train[0:train_size]
    response_train=response_train[0:train_size]
    
    sequence_test=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_seq_test.npy')
    response_test=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_res_test.npy')
    
    #Setting up the model
    import keras
    import numpy as np
    from keras import backend
    backend._BACKEND="theano"
    
    #DanQ model

    from keras.preprocessing import sequence
    from keras.optimizers import RMSprop
    from keras.models import Sequential
    from keras.layers.core import Dense
    from keras.layers.core import Merge
    from keras.layers.core import Dropout
    from keras.layers.core import Activation
    from keras.layers.core import Flatten
    from keras.layers.convolutional import Convolution1D, MaxPooling1D
    from keras.regularizers import l2, activity_l1
    from keras.constraints import maxnorm
    from keras.layers.recurrent import LSTM, GRU
    from keras.callbacks import ModelCheckpoint, EarlyStopping
    from keras.layers import Bidirectional

    model = Sequential()
    model.add(Convolution1D(nb_filter=20,filter_length=26,input_dim=4,input_length=201,border_mode="valid")) 
    model.add(Activation('relu'))


    model.add(MaxPooling1D(pool_length=6, stride=6))

    model.add(Dropout(0.2))

    model.add(Bidirectional(LSTM(5))) 
    model.add(Dropout(0.5))


    model.add(Dense(10))
    model.add(Activation('relu'))

    model.add(Dense(1))

    #compile the model
    model.compile(loss='mean_squared_error', optimizer='rmsprop')
    
    model.fit(sequence_train, response_train, validation_split=0.2,batch_size=100, nb_epoch=number_backpropagation_cycles, verbose=1)
    
    #evaulting correlation between model and test data
    import scipy

    correlation = scipy.stats.pearsonr(response_test,model.predict(sequence_test).flatten())
    correlation_2 = (correlation[0]**2)*100

    print('Percent of variability explained by model: '+str(correlation_2))
    
    # saving the model
    model.save('/mnt/Data_DapSeq_Maize/TrainedModel_DanQ_no_negative_' +ARF_label+'.h5')

In [5]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF27_smaller_GEM_events.txt','ARF27_smaller')

/usr/local/lib/python2.7/dist-packages/Bio/Seq.py:341: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [6]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF34_smaller_GEM_events.txt','ARF34_smaller')

In [7]:
Train_and_save_DanQ_model('ARF27_smaller',35)

Using TensorFlow backend.


Train on 39335 samples, validate on 9834 samples
Epoch 1/35
39335/39335 [==============================] - 78s - loss: 5413.9225 - val_loss: 3574.2081
Epoch 2/35
39335/39335 [==============================] - 77s - loss: 4828.2836 - val_loss: 3371.2509
Epoch 3/35
39335/39335 [==============================] - 76s - loss: 4700.8217 - val_loss: 3265.7315
Epoch 4/35
39335/39335 [==============================] - 77s - loss: 4657.4661 - val_loss: 3197.0838
Epoch 5/35
39335/39335 [==============================] - 76s - loss: 4548.6414 - val_loss: 3018.3132
Epoch 6/35
39335/39335 [==============================] - 76s - loss: 4383.2111 - val_loss: 3078.2806
Epoch 7/35
39335/39335 [==============================] - 76s - loss: 4279.4613 - val_loss: 2738.8646
Epoch 8/35
39335/39335 [==============================] - 76s - loss: 4150.0044 - val_loss: 2653.8492
Epoch 9/35
39335/39335 [==============================] - 76s - loss: 4099.5382 - val_loss: 2604.9077
Epoch 10/35
39335/39335 [========

In [8]:
Train_and_save_DanQ_model('ARF34_smaller',35)

Train on 95597 samples, validate on 23900 samples
Epoch 1/35
95597/95597 [==============================] - 191s - loss: 4887.8733 - val_loss: 4327.9871
Epoch 2/35
95597/95597 [==============================] - 190s - loss: 4531.3844 - val_loss: 4251.6972
Epoch 3/35
95597/95597 [==============================] - 190s - loss: 4456.6051 - val_loss: 4202.6710
Epoch 4/35
95597/95597 [==============================] - 187s - loss: 4241.0890 - val_loss: 3892.3050
Epoch 5/35
95597/95597 [==============================] - 186s - loss: 4007.0111 - val_loss: 3519.4219
Epoch 6/35
95597/95597 [==============================] - 187s - loss: 3848.9529 - val_loss: 3682.6917
Epoch 7/35
95597/95597 [==============================] - 186s - loss: 3752.3041 - val_loss: 3343.3430
Epoch 8/35
95597/95597 [==============================] - 187s - loss: 3653.3723 - val_loss: 3197.1191
Epoch 9/35
95597/95597 [==============================] - 187s - loss: 3575.5975 - val_loss: 3151.9682
Epoch 10/35
95597/95597

In [ ]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF16_GEM_events.txt','ARF16')

In [4]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF4_GEM_events.txt','ARF4')

/usr/local/lib/python2.7/dist-packages/Bio/Seq.py:341: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [4]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF4_rep2_GEM_events.txt','ARF4_rep2')

/usr/local/lib/python2.7/dist-packages/Bio/Seq.py:341: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [5]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF4_rep3_GEM_events.txt','ARF4_rep3')

In [5]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF10_GEM_events.txt','ARF10')

In [6]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF13_GEM_events.txt','ARF13')

In [7]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF18_GEM_events.txt','ARF18')

In [8]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF27_GEM_events.txt','ARF27')

In [9]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF29_GEM_events.txt','ARF29')

In [4]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF34_GEM_events.txt','ARF34')

/usr/local/lib/python2.7/dist-packages/Bio/Seq.py:341: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [5]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF35_GEM_events.txt','ARF35')

In [6]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF39_GEM_events.txt','ARF39')

In [4]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF10_rep1_ear_GEM_events.txt','ARF10_rep1_ear')

/usr/local/lib/python2.7/dist-packages/Bio/Seq.py:341: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [5]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF10_rep2_ear_GEM_events.txt','ARF10_rep2_ear')

In [6]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF10_rep1_tassel_GEM_events.txt','ARF10_rep1_tassel')

In [7]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF10_rep2_tassel_GEM_events.txt','ARF10_rep2_tassel')

In [5]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF7_GEM_events.txt','ARF7')

/usr/local/lib/python2.7/dist-packages/Bio/Seq.py:341: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [6]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF14_GEM_events.txt','ARF14')

In [7]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF24_GEM_events.txt','ARF24')

In [8]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF25_GEM_events.txt','ARF25')

In [9]:
Generate_training_and_test_datasets('/mnt/Data_DapSeq_Maize/ARF36_GEM_events.txt','ARF36')

In [10]:
Train_and_save_DanQ_model('ARF7',35)

Using TensorFlow backend.


Train on 37622 samples, validate on 9406 samples
Epoch 1/35
37622/37622 [==============================] - 75s - loss: 1642.5587 - val_loss: 1406.0280
Epoch 2/35
37622/37622 [==============================] - 73s - loss: 1336.9557 - val_loss: 1212.2301
Epoch 3/35
37622/37622 [==============================] - 73s - loss: 1176.2723 - val_loss: 1150.1994
Epoch 4/35
37622/37622 [==============================] - 73s - loss: 1108.3225 - val_loss: 1029.0691
Epoch 5/35
37622/37622 [==============================] - 73s - loss: 1028.1324 - val_loss: 947.3139
Epoch 6/35
37622/37622 [==============================] - 73s - loss: 965.0899 - val_loss: 875.0999
Epoch 7/35
37622/37622 [==============================] - 73s - loss: 901.5572 - val_loss: 870.0822
Epoch 8/35
37622/37622 [==============================] - 73s - loss: 856.4073 - val_loss: 826.4472
Epoch 9/35
37622/37622 [==============================] - 72s - loss: 819.4912 - val_loss: 783.1696
Epoch 10/35
37622/37622 [=================

In [11]:
Train_and_save_DanQ_model('ARF14',35)

Train on 35964 samples, validate on 8992 samples
Epoch 1/35
35964/35964 [==============================] - 74s - loss: 11727.8352 - val_loss: 10908.0097
Epoch 2/35
35964/35964 [==============================] - 74s - loss: 10596.9515 - val_loss: 9942.7811
Epoch 3/35
35964/35964 [==============================] - 73s - loss: 10001.3984 - val_loss: 9175.8203
Epoch 4/35
35964/35964 [==============================] - 73s - loss: 9506.0265 - val_loss: 8702.6930
Epoch 5/35
35964/35964 [==============================] - 73s - loss: 9059.9357 - val_loss: 8096.7627
Epoch 6/35
35964/35964 [==============================] - 73s - loss: 8630.9983 - val_loss: 7631.9033
Epoch 7/35
35964/35964 [==============================] - 72s - loss: 8143.9787 - val_loss: 7030.0634
Epoch 8/35
35964/35964 [==============================] - 72s - loss: 7883.4281 - val_loss: 7317.5209
Epoch 9/35
35964/35964 [==============================] - 72s - loss: 7467.0412 - val_loss: 6484.9748
Epoch 10/35
35964/35964 [====

In [12]:
Train_and_save_DanQ_model('ARF24',35)

Train on 6246 samples, validate on 1562 samples
Epoch 1/35
6246/6246 [==============================] - 13s - loss: 1845.6847 - val_loss: 1268.6567
Epoch 2/35
6246/6246 [==============================] - 13s - loss: 1752.4039 - val_loss: 1191.2670
Epoch 3/35
6246/6246 [==============================] - 13s - loss: 1664.3215 - val_loss: 1105.1476
Epoch 4/35
6246/6246 [==============================] - 13s - loss: 1575.9163 - val_loss: 1012.8874
Epoch 5/35
6246/6246 [==============================] - 13s - loss: 1481.7557 - val_loss: 927.2195
Epoch 6/35
6246/6246 [==============================] - 13s - loss: 1399.2385 - val_loss: 860.5576
Epoch 7/35
6246/6246 [==============================] - 13s - loss: 1359.1820 - val_loss: 819.0821
Epoch 8/35
6246/6246 [==============================] - 12s - loss: 1329.2209 - val_loss: 803.4981
Epoch 9/35
6246/6246 [==============================] - 13s - loss: 1322.7034 - val_loss: 799.4066
Epoch 10/35
6246/6246 [==============================] - 

In [13]:
Train_and_save_DanQ_model('ARF25',35)

Train on 49043 samples, validate on 12261 samples
Epoch 1/35
49043/49043 [==============================] - 100s - loss: 4231.9249 - val_loss: 3410.6344
Epoch 2/35
49043/49043 [==============================] - 99s - loss: 3469.8376 - val_loss: 2913.4720
Epoch 3/35
49043/49043 [==============================] - 99s - loss: 2966.6209 - val_loss: 2344.9716
Epoch 4/35
49043/49043 [==============================] - 99s - loss: 2545.7802 - val_loss: 2041.7218
Epoch 5/35
49043/49043 [==============================] - 99s - loss: 2342.5778 - val_loss: 1881.4029
Epoch 6/35
49043/49043 [==============================] - 99s - loss: 2248.5204 - val_loss: 1823.6334
Epoch 7/35
49043/49043 [==============================] - 100s - loss: 2156.5479 - val_loss: 1767.9331
Epoch 8/35
49043/49043 [==============================] - 99s - loss: 2086.5730 - val_loss: 1767.0500
Epoch 9/35
49043/49043 [==============================] - 98s - loss: 2031.4524 - val_loss: 1708.2211
Epoch 10/35
49043/49043 [=====

In [14]:
Train_and_save_DanQ_model('ARF36',35)

Train on 27740 samples, validate on 6936 samples
Epoch 1/35
27740/27740 [==============================] - 57s - loss: 2706.4706 - val_loss: 2243.1777
Epoch 2/35
27740/27740 [==============================] - 56s - loss: 2298.0900 - val_loss: 1827.3086
Epoch 3/35
27740/27740 [==============================] - 56s - loss: 1968.8071 - val_loss: 1543.3805
Epoch 4/35
27740/27740 [==============================] - 56s - loss: 1780.6917 - val_loss: 1374.7958
Epoch 5/35
27740/27740 [==============================] - 56s - loss: 1633.8010 - val_loss: 1190.2560
Epoch 6/35
27740/27740 [==============================] - 56s - loss: 1469.4236 - val_loss: 1080.1638
Epoch 7/35
27740/27740 [==============================] - 56s - loss: 1360.2485 - val_loss: 938.3718
Epoch 8/35
27740/27740 [==============================] - 57s - loss: 1276.4778 - val_loss: 1320.8241
Epoch 9/35
27740/27740 [==============================] - 56s - loss: 1222.6943 - val_loss: 962.0015
Epoch 10/35
27740/27740 [==========

In [8]:
Train_and_save_DanQ_model('ARF10_rep1_ear',35)

Using TensorFlow backend.


Train on 36341 samples, validate on 9086 samples
Epoch 1/35
36341/36341 [==============================] - 73s - loss: 2002.8636 - val_loss: 1671.7916
Epoch 2/35
36341/36341 [==============================] - 71s - loss: 1504.8027 - val_loss: 1196.3358
Epoch 3/35
36341/36341 [==============================] - 71s - loss: 1209.3262 - val_loss: 1019.3311
Epoch 4/35
36341/36341 [==============================] - 73s - loss: 1092.2851 - val_loss: 859.4826
Epoch 5/35
36341/36341 [==============================] - 72s - loss: 978.7255 - val_loss: 724.9203
Epoch 6/35
36341/36341 [==============================] - 72s - loss: 913.3822 - val_loss: 719.9395
Epoch 7/35
36341/36341 [==============================] - 71s - loss: 875.5293 - val_loss: 673.0429
Epoch 8/35
36341/36341 [==============================] - 72s - loss: 844.2221 - val_loss: 643.7188
Epoch 9/35
36341/36341 [==============================] - 72s - loss: 815.6348 - val_loss: 663.3058
Epoch 10/35
36341/36341 [===================

In [9]:
Train_and_save_DanQ_model('ARF10_rep2_ear',35)

Train on 53817 samples, validate on 13455 samples
Epoch 1/35
53817/53817 [==============================] - 105s - loss: 2735.9892 - val_loss: 2196.0429
Epoch 2/35
53817/53817 [==============================] - 104s - loss: 2074.7095 - val_loss: 1679.8025
Epoch 3/35
53817/53817 [==============================] - 105s - loss: 1823.5133 - val_loss: 1452.7318
Epoch 4/35
53817/53817 [==============================] - 106s - loss: 1548.3769 - val_loss: 1139.5833
Epoch 5/35
53817/53817 [==============================] - 107s - loss: 1368.5312 - val_loss: 1367.0449
Epoch 6/35
53817/53817 [==============================] - 106s - loss: 1270.4029 - val_loss: 956.2281
Epoch 7/35
53817/53817 [==============================] - 107s - loss: 1224.3185 - val_loss: 918.1251
Epoch 8/35
53817/53817 [==============================] - 107s - loss: 1185.7815 - val_loss: 918.5137
Epoch 9/35
53817/53817 [==============================] - 106s - loss: 1151.5525 - val_loss: 851.4569
Epoch 10/35
53817/53817 [==

In [10]:
Train_and_save_DanQ_model('ARF10_rep1_tassel',35)

Train on 31403 samples, validate on 7851 samples
Epoch 1/35
31403/31403 [==============================] - 63s - loss: 1571.1728 - val_loss: 1351.4851
Epoch 2/35
31403/31403 [==============================] - 63s - loss: 1196.3053 - val_loss: 1037.7606
Epoch 3/35
31403/31403 [==============================] - 63s - loss: 1016.0788 - val_loss: 942.9680
Epoch 4/35
31403/31403 [==============================] - 63s - loss: 960.3343 - val_loss: 886.9431
Epoch 5/35
31403/31403 [==============================] - 62s - loss: 910.0408 - val_loss: 816.6931
Epoch 6/35
31403/31403 [==============================] - 62s - loss: 824.9694 - val_loss: 733.0281
Epoch 7/35
31403/31403 [==============================] - 62s - loss: 772.4587 - val_loss: 670.1912
Epoch 8/35
31403/31403 [==============================] - 62s - loss: 722.3292 - val_loss: 1066.9985
Epoch 9/35
31403/31403 [==============================] - 62s - loss: 696.5746 - val_loss: 757.6237
Epoch 10/35
31403/31403 [====================

In [11]:
Train_and_save_DanQ_model('ARF10_rep2_tassel',35)

Train on 48745 samples, validate on 12187 samples
Epoch 1/35
48745/48745 [==============================] - 99s - loss: 2912.2922 - val_loss: 2496.6601
Epoch 2/35
48745/48745 [==============================] - 97s - loss: 2320.6934 - val_loss: 2059.1017
Epoch 3/35
48745/48745 [==============================] - 97s - loss: 2077.2247 - val_loss: 1963.1966
Epoch 4/35
48745/48745 [==============================] - 99s - loss: 1984.6060 - val_loss: 1876.5615
Epoch 5/35
48745/48745 [==============================] - 97s - loss: 1842.2177 - val_loss: 1759.1762
Epoch 6/35
48745/48745 [==============================] - 94s - loss: 1674.5266 - val_loss: 1510.3170
Epoch 7/35
48745/48745 [==============================] - 94s - loss: 1564.8009 - val_loss: 1660.5494
Epoch 8/35
48745/48745 [==============================] - 94s - loss: 1506.3860 - val_loss: 1413.6896
Epoch 9/35
48745/48745 [==============================] - 93s - loss: 1442.3534 - val_loss: 1337.6433
Epoch 10/35
48745/48745 [=======

In [6]:
Train_and_save_DanQ_model('ARF4',35)

Using TensorFlow backend.


Train on 36776 samples, validate on 9195 samples
Epoch 1/35
36776/36776 [==============================] - 73s - loss: 2127.7159 - val_loss: 1398.1829
Epoch 2/35
36776/36776 [==============================] - 70s - loss: 1752.4591 - val_loss: 1306.4759
Epoch 3/35
36776/36776 [==============================] - 70s - loss: 1740.4532 - val_loss: 1299.6157
Epoch 4/35
36776/36776 [==============================] - 69s - loss: 1711.7987 - val_loss: 1246.7413
Epoch 5/35
36776/36776 [==============================] - 68s - loss: 1670.8104 - val_loss: 1206.6119
Epoch 6/35
36776/36776 [==============================] - 68s - loss: 1615.2978 - val_loss: 1275.9449
Epoch 7/35
36776/36776 [==============================] - 68s - loss: 1516.6498 - val_loss: 954.5047
Epoch 8/35
36776/36776 [==============================] - 68s - loss: 1433.8557 - val_loss: 1005.2452
Epoch 9/35
36776/36776 [==============================] - 68s - loss: 1382.1148 - val_loss: 1381.7839
Epoch 10/35
36776/36776 [=========

In [7]:
Train_and_save_DanQ_model('ARF4_rep2',35)

Train on 19473 samples, validate on 4869 samples
Epoch 1/35
19473/19473 [==============================] - 37s - loss: 1679.4559 - val_loss: 1132.3133
Epoch 2/35
19473/19473 [==============================] - 36s - loss: 1383.0800 - val_loss: 931.4516
Epoch 3/35
19473/19473 [==============================] - 36s - loss: 1301.4132 - val_loss: 913.7392
Epoch 4/35
19473/19473 [==============================] - 36s - loss: 1290.4728 - val_loss: 912.5556
Epoch 5/35
19473/19473 [==============================] - 36s - loss: 1277.8503 - val_loss: 865.1717
Epoch 6/35
19473/19473 [==============================] - 36s - loss: 1282.8576 - val_loss: 891.8963
Epoch 7/35
19473/19473 [==============================] - 36s - loss: 1272.3135 - val_loss: 871.9457
Epoch 8/35
19473/19473 [==============================] - 36s - loss: 1244.8601 - val_loss: 861.7561
Epoch 9/35
19473/19473 [==============================] - 36s - loss: 1216.9461 - val_loss: 803.5947
Epoch 10/35
19473/19473 [================

In [7]:
Train_and_save_DanQ_model('ARF4_rep3',35)

Train on 36190 samples, validate on 9048 samples
Epoch 1/35
36190/36190 [==============================] - 72s - loss: 3367.6591 - val_loss: 3307.8301
Epoch 2/35
36190/36190 [==============================] - 71s - loss: 2849.5019 - val_loss: 3014.9194
Epoch 3/35
36190/36190 [==============================] - 71s - loss: 2779.7190 - val_loss: 3007.9033
Epoch 4/35
36190/36190 [==============================] - 69s - loss: 2758.3360 - val_loss: 3007.5720
Epoch 5/35
36190/36190 [==============================] - 67s - loss: 2726.9250 - val_loss: 2967.4282
Epoch 6/35
36190/36190 [==============================] - 67s - loss: 2714.6290 - val_loss: 2896.4135
Epoch 7/35
36190/36190 [==============================] - 67s - loss: 2638.8910 - val_loss: 2822.2334
Epoch 8/35
36190/36190 [==============================] - 67s - loss: 2562.6571 - val_loss: 2736.8975
Epoch 9/35
36190/36190 [==============================] - 67s - loss: 2480.9359 - val_loss: 2673.9643
Epoch 10/35
36190/36190 [========

In [8]:
Train_and_save_DanQ_model('ARF10',35)

Train on 42963 samples, validate on 10741 samples
Epoch 1/35
42963/42963 [==============================] - 82s - loss: 2364.4806 - val_loss: 2013.0646
Epoch 2/35
42963/42963 [==============================] - 81s - loss: 1734.5396 - val_loss: 1463.8579
Epoch 3/35
42963/42963 [==============================] - 81s - loss: 1371.7000 - val_loss: 1130.8974
Epoch 4/35
42963/42963 [==============================] - 82s - loss: 1180.5277 - val_loss: 1069.9976
Epoch 5/35
42963/42963 [==============================] - 81s - loss: 1087.6656 - val_loss: 932.7324
Epoch 6/35
42963/42963 [==============================] - 80s - loss: 1046.5845 - val_loss: 866.3762
Epoch 7/35
42963/42963 [==============================] - 80s - loss: 1008.9000 - val_loss: 827.6527
Epoch 8/35
42963/42963 [==============================] - 79s - loss: 970.7698 - val_loss: 844.0327
Epoch 9/35
42963/42963 [==============================] - 79s - loss: 946.1754 - val_loss: 806.4745
Epoch 10/35
42963/42963 [==============

In [9]:
Train_and_save_DanQ_model('ARF13',35)

Train on 23500 samples, validate on 5876 samples
Epoch 1/35
23500/23500 [==============================] - 45s - loss: 3260.6062 - val_loss: 2878.4745
Epoch 2/35
23500/23500 [==============================] - 45s - loss: 2730.0433 - val_loss: 2466.1536
Epoch 3/35
23500/23500 [==============================] - 45s - loss: 2282.1756 - val_loss: 1842.5053
Epoch 4/35
23500/23500 [==============================] - 45s - loss: 1844.0153 - val_loss: 1406.7536
Epoch 5/35
23500/23500 [==============================] - 45s - loss: 1541.0483 - val_loss: 1129.5317
Epoch 6/35
23500/23500 [==============================] - 45s - loss: 1379.6999 - val_loss: 1061.9651
Epoch 7/35
23500/23500 [==============================] - 45s - loss: 1297.5732 - val_loss: 1030.1212
Epoch 8/35
23500/23500 [==============================] - 45s - loss: 1242.7591 - val_loss: 913.2825
Epoch 9/35
23500/23500 [==============================] - 44s - loss: 1189.4518 - val_loss: 908.1424
Epoch 10/35
23500/23500 [==========

In [10]:
Train_and_save_DanQ_model('ARF16',35)

Train on 53941 samples, validate on 13486 samples
Epoch 1/35
53941/53941 [==============================] - 104s - loss: 9463.3411 - val_loss: 11466.4891
Epoch 2/35
53941/53941 [==============================] - 103s - loss: 8558.3027 - val_loss: 11119.7556
Epoch 3/35
53941/53941 [==============================] - 103s - loss: 8424.2766 - val_loss: 10964.1154
Epoch 4/35
53941/53941 [==============================] - 103s - loss: 8299.6816 - val_loss: 10780.1169
Epoch 5/35
53941/53941 [==============================] - 103s - loss: 8038.9400 - val_loss: 10451.6659
Epoch 6/35
53941/53941 [==============================] - 103s - loss: 7760.9728 - val_loss: 10418.1807
Epoch 7/35
53941/53941 [==============================] - 102s - loss: 7485.4884 - val_loss: 10147.1557
Epoch 8/35
53941/53941 [==============================] - 102s - loss: 7339.0710 - val_loss: 10459.5790
Epoch 9/35
53941/53941 [==============================] - 103s - loss: 7212.2982 - val_loss: 9553.9941
Epoch 10/35
539

In [11]:
Train_and_save_DanQ_model('ARF18',35)

Train on 9536 samples, validate on 2384 samples
Epoch 1/35
9536/9536 [==============================] - 19s - loss: 4551.3087 - val_loss: 3765.3687
Epoch 2/35
9536/9536 [==============================] - 18s - loss: 4264.2203 - val_loss: 3494.2492
Epoch 3/35
9536/9536 [==============================] - 18s - loss: 3976.0749 - val_loss: 3214.3347
Epoch 4/35
9536/9536 [==============================] - 18s - loss: 3750.8471 - val_loss: 3001.1992
Epoch 5/35
9536/9536 [==============================] - 18s - loss: 3623.5280 - val_loss: 2897.0141
Epoch 6/35
9536/9536 [==============================] - 18s - loss: 3576.6640 - val_loss: 2878.4662
Epoch 7/35
9536/9536 [==============================] - 18s - loss: 3555.4869 - val_loss: 2872.2026
Epoch 8/35
9536/9536 [==============================] - 18s - loss: 3559.6610 - val_loss: 2871.8774
Epoch 9/35
9536/9536 [==============================] - 18s - loss: 3544.3018 - val_loss: 2865.2712
Epoch 10/35
9536/9536 [=============================

In [12]:
Train_and_save_DanQ_model('ARF27',35)

Train on 76796 samples, validate on 19200 samples
Epoch 1/35
76796/76796 [==============================] - 147s - loss: 11567.5999 - val_loss: 7797.8566
Epoch 2/35
76796/76796 [==============================] - 146s - loss: 10485.7554 - val_loss: 7440.0080
Epoch 3/35
76796/76796 [==============================] - 147s - loss: 10331.7848 - val_loss: 7394.0962
Epoch 4/35
76796/76796 [==============================] - 146s - loss: 10100.9652 - val_loss: 7577.4496
Epoch 5/35
76796/76796 [==============================] - 146s - loss: 9793.3170 - val_loss: 6915.0043
Epoch 6/35
76796/76796 [==============================] - 146s - loss: 9502.8091 - val_loss: 6240.1487
Epoch 7/35
76796/76796 [==============================] - 146s - loss: 9284.4717 - val_loss: 6883.5863
Epoch 8/35
76796/76796 [==============================] - 146s - loss: 9105.8293 - val_loss: 6142.5660
Epoch 9/35
76796/76796 [==============================] - 146s - loss: 8907.7040 - val_loss: 5788.8614
Epoch 10/35
76796/7

In [ ]:
Train_and_save_DanQ_model('ARF29',35)

Train on 18176 samples, validate on 4545 samples
Epoch 1/35
18176/18176 [==============================] - 36s - loss: 4012.5159 - val_loss: 7084.5936
Epoch 2/35
18176/18176 [==============================] - 34s - loss: 3566.8719 - val_loss: 6579.0753
Epoch 3/35
18176/18176 [==============================] - 34s - loss: 3286.6815 - val_loss: 6295.5785
Epoch 4/35
18176/18176 [==============================] - 34s - loss: 3247.5279 - val_loss: 6296.0166
Epoch 5/35
18176/18176 [==============================] - 34s - loss: 3208.5870 - val_loss: 6249.1865
Epoch 6/35
18176/18176 [==============================] - 34s - loss: 3190.5696 - val_loss: 6212.4715
Epoch 7/35
18176/18176 [==============================] - 34s - loss: 3186.3926 - val_loss: 6198.6038
Epoch 8/35
18176/18176 [==============================] - 35s - loss: 3160.3990 - val_loss: 6161.8641
Epoch 9/35
18176/18176 [==============================] - 34s - loss: 3126.4145 - val_loss: 6011.4645
Epoch 10/35
18176/18176 [========

In [4]:
Train_and_save_DanQ_model('ARF34',35)

Using TensorFlow backend.


Train on 143977 samples, validate on 35995 samples
Epoch 1/35
143977/143977 [==============================] - 285s - loss: 7579.5630 - val_loss: 6716.8616
Epoch 2/35
143977/143977 [==============================] - 283s - loss: 6900.0216 - val_loss: 6101.3160
Epoch 3/35
143977/143977 [==============================] - 283s - loss: 6460.5100 - val_loss: 5660.7465
Epoch 4/35
143977/143977 [==============================] - 279s - loss: 6161.7054 - val_loss: 5412.6841
Epoch 5/35
143977/143977 [==============================] - 278s - loss: 5891.0374 - val_loss: 5073.5241
Epoch 6/35
143977/143977 [==============================] - 273s - loss: 5677.0033 - val_loss: 5209.0427
Epoch 7/35
143977/143977 [==============================] - 272s - loss: 5523.8373 - val_loss: 5087.7659
Epoch 8/35
143977/143977 [==============================] - 272s - loss: 5443.0108 - val_loss: 4654.2568
Epoch 9/35
143977/143977 [==============================] - 272s - loss: 5378.5422 - val_loss: 4784.9655
Epoc

In [5]:
Train_and_save_DanQ_model('ARF35',35)

Train on 11855 samples, validate on 2964 samples
Epoch 1/35
11855/11855 [==============================] - 24s - loss: 6097.4915 - val_loss: 5784.4525
Epoch 2/35
11855/11855 [==============================] - 23s - loss: 5772.8231 - val_loss: 5463.0940
Epoch 3/35
11855/11855 [==============================] - 23s - loss: 5440.4093 - val_loss: 5099.5840
Epoch 4/35
11855/11855 [==============================] - 23s - loss: 5102.9008 - val_loss: 4760.2303
Epoch 5/35
11855/11855 [==============================] - 23s - loss: 4855.0404 - val_loss: 4536.7182
Epoch 6/35
11855/11855 [==============================] - 23s - loss: 4734.2198 - val_loss: 4467.9315
Epoch 7/35
11855/11855 [==============================] - 23s - loss: 4741.3306 - val_loss: 4454.6128
Epoch 8/35
11855/11855 [==============================] - 23s - loss: 4697.8945 - val_loss: 4430.3923
Epoch 9/35
11855/11855 [==============================] - 23s - loss: 4688.1672 - val_loss: 4399.4323
Epoch 10/35
11855/11855 [========

In [6]:
Train_and_save_DanQ_model('ARF39',35)

Train on 13050 samples, validate on 3263 samples
Epoch 1/35
13050/13050 [==============================] - 26s - loss: 7428.7201 - val_loss: 7121.2714
Epoch 2/35
13050/13050 [==============================] - 25s - loss: 6905.6091 - val_loss: 6518.6309
Epoch 3/35
13050/13050 [==============================] - 25s - loss: 6401.2708 - val_loss: 6051.2632
Epoch 4/35
13050/13050 [==============================] - 25s - loss: 6067.2285 - val_loss: 5770.6936
Epoch 5/35
13050/13050 [==============================] - 25s - loss: 5732.0044 - val_loss: 5313.6510
Epoch 6/35
13050/13050 [==============================] - 25s - loss: 5421.6541 - val_loss: 4916.7067
Epoch 7/35
13050/13050 [==============================] - 26s - loss: 5175.1188 - val_loss: 4663.0209
Epoch 8/35
13050/13050 [==============================] - 25s - loss: 4952.7396 - val_loss: 4365.2324
Epoch 9/35
13050/13050 [==============================] - 25s - loss: 4755.1789 - val_loss: 4247.4092
Epoch 10/35
13050/13050 [========

# Creating dataset without a negative set

In [4]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF4_GEM_events.txt','ARF4')

/usr/local/lib/python2.7/dist-packages/Bio/Seq.py:341: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [5]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF39_GEM_events.txt','ARF39')

In [6]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF35_GEM_events.txt','ARF35')

In [7]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF34_GEM_events.txt','ARF34')

In [8]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF10_GEM_events.txt','ARF10')

In [9]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF13_GEM_events.txt','ARF13')

In [10]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF16_GEM_events.txt','ARF16')

In [11]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF18_GEM_events.txt','ARF18')

In [12]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF27_GEM_events.txt','ARF27')

In [14]:
Generate_training_and_test_datasets_no_negative('/mnt/Data_DapSeq_Maize/ARF29_GEM_events.txt','ARF29')

In [12]:
#finding the min length of the test set
List_of_ARFs =['ARF4','ARF10','ARF13','ARF16','ARF18','ARF27','ARF29','ARF34','ARF35','ARF39']
seq_test_sets = [None]*len(List_of_ARFs)

counter1=0
for ARF_label in List_of_ARFs:
    seq_test_sets[counter1]=numpy.load('/mnt/Data_DapSeq_Maize/'+ARF_label+'no_negative_seq_test.npy')
    print(len(seq_test_sets[counter1]))
    counter1=counter1+1

#based on this the test set will only be: 5960 in size


5747
6713
3672
8429
1490
12000
2841
22497
1853
2040


In [5]:
Train_and_save_DanQ_model_no_negative('ARF4',35,5960)

Using TensorFlow backend.


Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 5890.5125 - val_loss: 3614.3051
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 5528.7237 - val_loss: 3294.9960
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 5235.2277 - val_loss: 3047.9870
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 4984.3632 - val_loss: 2803.1345
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 4736.9652 - val_loss: 2548.9849
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 4460.6861 - val_loss: 2294.1966
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 4191.2999 - val_loss: 2046.2299
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 3953.9948 - val_loss: 1813.1747
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 3738.8059 - val_loss: 1598.4614
Epoch 10/35
4768/4768 [==============================] - 9s -

In [6]:
Train_and_save_DanQ_model_no_negative('ARF39',35,5960)

Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 14577.9416 - val_loss: 12933.7465
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 14073.1560 - val_loss: 12504.9977
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 13627.9211 - val_loss: 12106.2705
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 13168.9152 - val_loss: 11632.3015
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 12722.4319 - val_loss: 11137.6340
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 12167.5643 - val_loss: 10612.4973
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 11630.5282 - val_loss: 10079.3457
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 11041.3092 - val_loss: 9539.3172
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 10532.2490 - val_loss: 9018.0391
Epoch 10/35
4768/4768 [======================

In [7]:
Train_and_save_DanQ_model_no_negative('ARF35',35,5960)

Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 10946.4969 - val_loss: 11185.2873
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 10533.5791 - val_loss: 10834.7799
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 10172.1700 - val_loss: 10463.8320
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 9768.0352 - val_loss: 10055.2487
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 9382.1408 - val_loss: 9659.6886
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 8987.8170 - val_loss: 9230.7290
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 8547.3093 - val_loss: 8785.8348
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 8088.5609 - val_loss: 8313.3311
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 7666.6547 - val_loss: 7840.2453
Epoch 10/35
4768/4768 [==============================]

In [8]:
Train_and_save_DanQ_model_no_negative('ARF34',35,5960)

Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 17719.5891 - val_loss: 15691.4057
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 17452.0443 - val_loss: 15405.6035
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 17168.2036 - val_loss: 15148.4330
Epoch 4/35
4768/4768 [==============================] - 8s - loss: 16901.5397 - val_loss: 14895.5741
Epoch 5/35
4768/4768 [==============================] - 8s - loss: 16628.1404 - val_loss: 14619.2694
Epoch 6/35
4768/4768 [==============================] - 8s - loss: 16361.8708 - val_loss: 14326.8482
Epoch 7/35
4768/4768 [==============================] - 8s - loss: 16051.8572 - val_loss: 14009.0199
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 15706.0098 - val_loss: 13676.6697
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 15397.3244 - val_loss: 13333.9494
Epoch 10/35
4768/4768 [====================

In [6]:
Train_and_save_DanQ_model_no_negative('ARF10',35,5960)

Using TensorFlow backend.


Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 10s - loss: 6038.4743 - val_loss: 5441.5113
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 6013.6423 - val_loss: 5409.8628
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 5957.5842 - val_loss: 5327.1531
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 5844.2286 - val_loss: 5188.5432
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 5695.9725 - val_loss: 5036.9625
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 5543.7034 - val_loss: 4894.5348
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 5386.9951 - val_loss: 4749.1138
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 5228.7694 - val_loss: 4593.0795
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 5060.7379 - val_loss: 4424.5420
Epoch 10/35
4768/4768 [==============================] - 9s 

In [7]:
Train_and_save_DanQ_model_no_negative('ARF13',35,5960)

Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 7162.8012 - val_loss: 7485.3040
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 6905.1397 - val_loss: 7258.5765
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 6696.2349 - val_loss: 7051.5224
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 6462.5410 - val_loss: 6790.3744
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 6218.5152 - val_loss: 6525.9009
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 5944.5448 - val_loss: 6231.9420
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 5665.5727 - val_loss: 5918.8718
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 5383.3247 - val_loss: 5582.5464
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 5059.8562 - val_loss: 5234.5013
Epoch 10/35
4768/4768 [==============================] - 9s -

In [8]:
Train_and_save_DanQ_model_no_negative('ARF16',35,5960)

Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 19244.4914 - val_loss: 24385.7134
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 18528.0734 - val_loss: 23616.8646
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 17900.6331 - val_loss: 23054.1207
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 17397.3471 - val_loss: 22536.2846
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 16954.8024 - val_loss: 21986.4600
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 16358.7515 - val_loss: 21410.0441
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 15858.3919 - val_loss: 20821.5654
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 15400.5026 - val_loss: 20228.1970
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 14880.9121 - val_loss: 19647.1964
Epoch 10/35
4768/4768 [====================

In [9]:
Train_and_save_DanQ_model_no_negative('ARF18',35,5960)

Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 8748.9681 - val_loss: 8777.8464
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 8583.0339 - val_loss: 8566.4760
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 8386.2066 - val_loss: 8387.5742
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 8198.7488 - val_loss: 8203.5701
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 7990.8369 - val_loss: 7997.7229
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 7784.3298 - val_loss: 7772.3799
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 7540.8908 - val_loss: 7531.1038
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 7290.9073 - val_loss: 7271.7973
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 7025.3031 - val_loss: 7002.6395
Epoch 10/35
4768/4768 [==============================] - 9s -

In [10]:
Train_and_save_DanQ_model_no_negative('ARF27',35,5960)

Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 18744.2722 - val_loss: 19074.3203
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 18279.2641 - val_loss: 18718.4480
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 17938.3101 - val_loss: 18393.8900
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 17556.0541 - val_loss: 18022.5358
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 17171.9077 - val_loss: 17615.0990
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 16740.2670 - val_loss: 17142.5259
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 16288.3951 - val_loss: 16642.7629
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 15763.2628 - val_loss: 16102.7216
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 15222.8111 - val_loss: 15543.7195
Epoch 10/35
4768/4768 [====================

In [11]:
Train_and_save_DanQ_model_no_negative('ARF29',35,5960)

Train on 4768 samples, validate on 1192 samples
Epoch 1/35
4768/4768 [==============================] - 9s - loss: 10420.5704 - val_loss: 8756.7509
Epoch 2/35
4768/4768 [==============================] - 9s - loss: 10109.8713 - val_loss: 8375.7107
Epoch 3/35
4768/4768 [==============================] - 9s - loss: 9781.5750 - val_loss: 8090.5768
Epoch 4/35
4768/4768 [==============================] - 9s - loss: 9485.4790 - val_loss: 7809.4614
Epoch 5/35
4768/4768 [==============================] - 9s - loss: 9189.2579 - val_loss: 7515.7224
Epoch 6/35
4768/4768 [==============================] - 9s - loss: 8905.5076 - val_loss: 7205.1788
Epoch 7/35
4768/4768 [==============================] - 9s - loss: 8594.0850 - val_loss: 6881.0137
Epoch 8/35
4768/4768 [==============================] - 9s - loss: 8260.0268 - val_loss: 6493.9966
Epoch 9/35
4768/4768 [==============================] - 9s - loss: 7872.9390 - val_loss: 6102.5359
Epoch 10/35
4768/4768 [==============================] - 9s